<a href="https://colab.research.google.com/github/wizard339/education/blob/main/transformers_text_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Загрузка датасета

Загрузим файл с датасетом и посмотрим, какие тексты он содержит. Скачать файл вы можете на учебной платформе.

В качестве данных для начала фразы мы будем использовать небольшую выборку из твитов твиттер-канала “Усы Пескова”. Берутся 7 произвольных твитов, разделенных специальным знаком, и на основе этого текста тренированная модель дописывает продолжения. 


Как и в предыдущем примере с использованием RNN, одним из важных параметров является температура. Однако, тут также есть ряд других настраиваемых параметров.

In [1]:
!pip install transformers > /dev/null

In [2]:
with open('tweets.txt', 'r') as f:
    tweets = f.read().strip().split('\n\n')
print(len(tweets))
for i in range(3):
    print(tweets[i])

26
Соловьев наконец-то вышел на новый уровень - теперь его стали банить и в офлайне
Дарим мы тебе бутылку игристого вина. Пить тебе еще рано, но встретиться с ней за некоторые преступления ты уже можешь. ПОЗ-ДРАВ-ЛЯ-ЕМ!
Да. Еще очень многие помнят, что такое госплан, как планировалось, талоны на еду, очереди, дефицит, выездные визы. Но спасибо, что напомнил


In [3]:
import torch

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
device

device(type='cuda')

In [4]:
from transformers import AutoTokenizer, AutoModelForCausalLM

# Загрузка модели

Рассмотрим генерацию текста с использованием предобученных моделей. Ранее мы рассмотрели, что их есть небольшое количество, даже для русского языка. 

В рамках этого ноутбука будет использоваться самая маленькая нейросеть ruGPT-3 small на основе GPT-2, тренировавшаяся на корпусах текста русского языка. Эта модель является авторегрессионной, то есть она учитывает только контекст слева при генерации продолжения текста.


In [5]:
model_name = 'sberbank-ai/rugpt3small_based_on_gpt2'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


# Генерация текста

In [6]:
import random

Возьмем произвольные твиты, на которых будет тренироваться модель:

In [7]:
sep = '\n***\n'

prefix = sep.join([''] + random.sample(tweets, k=7) + [''])

tokens = tokenizer(prefix, return_tensors='pt')
tokens = {k: v.to(model.device) for k, v in tokens.items()}
end_token_id = tokenizer.encode('*')[0]
print(prefix)


***
Теперь главное, чтобы все дожили до второго чтения
***
Есть подозрение, что те самые 400 кг порошка хранятся в кабинете Володина под его чутким контролем. Иначе эти фантазии не объяснить
***
Внимание. В России официально появился парк славы Илона Маска
***
Но тут главное, чтобы тракторы и бульдозеры были в порядке - и продолжали качественно давить вредные иностранные продукты
***
Ну это примерно как Пескову поручить расследовать информацию о его часах
***
Знаете, это тот случай, когда Россия может на 400% доверять своему партнеру
***
В сеть утекли кадры с репетиции следующей инаугурации Путина
***



Посмотрим, что генерирует модель:

In [9]:
size = tokens['input_ids'].shape[1]
output = model.generate(
    **tokens, 
    do_sample=True, 
    max_length=size+128, 
    repetition_penalty=1.2, 
    temperature=0.5,
    num_beams=1,
    pad_token_id=tokenizer.eos_token_id
)
decoded = tokenizer.decode(output[0])
result = decoded[len(prefix):]
print('\n', result)


 На фоне тех же событий по поводу выборов в Госдуму РФ
***
Чего только стоит выступление Медведева перед избирателями: &quot;Я проголосую за вас&quot;, &laquo;Давайте сделаем так, чтобы выборы прошли честно&raquo;.
***
А вот еще один эпизод из жизни нашего президента. Он уже третий год подряд является президентом страны. За последние 15 лет он ни разу не проиграл. Это значит, что мы можем спокойно смотреть на то, как Путин делает свою работу. А если бы он проигрывал? Или опять пошел бы к власти? Что тогда было бы?
***
Зато сейчас


# Чат-бот

Одно из самых удивительных применений такой модели является то, что можно достаточно лаконично встроить ее и создать разговорный чат-бот, который будет придумывать ответы, опираясь на ваш предыдущий контекст разговора.

In [10]:
def respond_to_dialog(texts):
    prefix = '\nx:'
    for i, t in enumerate(texts):
        prefix += t
        prefix += '\nx:' if i % 2 == 1 else '\ny:'
    tokens = tokenizer(prefix, return_tensors='pt')
    tokens = {k: v.to(model.device) for k, v in tokens.items()}
    end_token_id = tokenizer.encode('\n')[0]
    size = tokens['input_ids'].shape[1]
    output = model.generate(
        **tokens, 
        eos_token_id=end_token_id,
        do_sample=True, 
        max_length=size+128, 
        repetition_penalty=3.2, 
        temperature=1,
        num_beams=3,
        length_penalty=0.01,
        pad_token_id=tokenizer.eos_token_id
    )
    decoded = tokenizer.decode(output[0])
    result = decoded[len(prefix):]
    return result.strip()

In [12]:
seed = input('Начните диалог с ботом любой фразой\n')
history = [seed]
while True:
    result = respond_to_dialog(history[-10:])
    next_sentence = input(result + '\n')
    history.append(result)
    history.append(next_sentence)

Начните диалог с ботом любой фразой
Привет, как жизнь?
У меня всё хорошо.
У меня тоже, только вот на работе устаю в последнее время.
А у тебя?
У меня тоже, только вот на работе устаю в последнее время.
А у тебя?
У меня тоже все хорошо, только вот на работе устаю в последнее время.
А у тебя?
и у меня
А у тебя?
тебя заклинило что ли?
а у меня нет
вот ты шутник
ну я пошутил
я заметил
ты не шучу
а я не шучу
это нормально
согласен


KeyboardInterrupt: ignored

In [13]:
for i in range(10):
    print(respond_to_dialog(['Давай поговорим о домашних животных', 'Каких питомцев вы любите?\n--------', 
                             'Давай поговорим о машинах', 'Какого цвета твой автомобиль?\n--------',
                             'Давай поговорим о физике']))

Какое животное ты любишь?
Какие предметы ты любишь?
Как ты относишься к тому, чтобы быть умным?
Какие есть недостатки у машины?
Какое расстояние от Земли до Луны?
Какое расстояние от Земли до Луны?
Как ты думаешь, что такое электричество?
Какое расстояние от Земли до Луны?
Какие у тебя любимые животные?
Какое животное ты любишь?
